In [ ]:
import cohere
co = cohere.Client('<<apiKey>>')

docs = ['Carson City is the capital city of the American state of Nevada.',
'The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean. Its capital is Saipan.',
'Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district.',
'Capital punishment (the death penalty) has existed in the United States since beforethe United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.']

response = co.rerank(
  model = 'rerank-english-v2.0',
  query = 'What is the capital of the United States?',
  documents = docs,
  top_n = 3,
)
print(response)

In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
from cohere import Client

csv_file_path = "C:\\Users\\annem\\OneDrive\\Desktop\\ohdsi concepts\\CONCEPT.csv"
output_csv_file_path = r"C:\Users\annem\OneDrive\Desktop\conceptsNormalization\embeddings3.csv"

df = pd.read_csv(csv_file_path, sep='\t', quoting=3, header=None).head(100)
df.columns = ["concept_id", "concept_name", "domain_id", "vocabulary_id", "concept_class_id", "standard_concept", "concept_code", "valid_start_date", "valid_end_date", "invalid_reason"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

loaded_embeddings_df = pd.read_csv(output_csv_file_path)
desired_domain_id = "Provider"

filtered_df = df[df['domain_id'] == desired_domain_id]

if not filtered_df.empty:
    filtered_embeddings = [torch.tensor(eval(embedding)) for embedding in loaded_embeddings_df[loaded_embeddings_df['concept_id'].isin(filtered_df['concept_id'])]['embedding']]
    filtered_embeddings = torch.stack(filtered_embeddings)
    
    query = "lobes of the lung"
    query=str(query)
    query_embedding = model.encode(query)
    query_embedding = torch.tensor(query_embedding)

    cosine_similarity = torch.nn.functional.cosine_similarity(query_embedding, filtered_embeddings)
    top_10_indices = torch.topk(cosine_similarity, k=10).indices

    top_10_results = loaded_embeddings_df.iloc[top_10_indices]

    if not top_10_results.empty:
        matching_rows = filtered_df[filtered_df['concept_id'].isin(top_10_results['concept_id'].values)]

        if not matching_rows.empty:
            # Use cohere for reranking
            cohere_api_key = "ium8JWus9GHqhQ0huVEPk2L6zw27zpyhUHUh7AN1"
            cohere_client = Client(api_key=cohere_api_key)

            documents = [{"text": content} for content in matching_rows['concept_name'].values]
            #query = {"text": query}

            # Rerank using cohere
            reranked_docs = cohere_client.rerank(model="embed-english-v3.0", query=query, documents=documents, top_n=1)
            top_reranked_concept_name = reranked_docs[0]['text']
            
            print(f"Top Reranked Concept Name for Domain {desired_domain_id}: {top_reranked_concept_name}")

        else:
            print(f"No matching rows found in the DataFrame for the top 10 similarity concept IDs in Domain {desired_domain_id}.")
    else:
        print(f"No rows found in the top 10 results.")
else:
    print(f"No rows found in the DataFrame for Domain {desired_domain_id}.")


C:\Users\annem\AppData\Local\Temp\ipykernel_6920\976998926.py:9: DtypeWarning: Columns (0,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path, sep='\t', quoting=3, header=None).head(100)


CohereAPIError: internal server error, this has been reported to our developers. id 735bc272-07aa-46ce-8f80-1d4810e7ce99

In [7]:
import pandas as pd

# Read the Excel file
df = pd.read_excel(r"C:\Users\annem\OneDrive\Desktop\conceptsNormalization\all_criteria_corpus.xlsx")

# Remove duplicates
df.drop_duplicates(inplace=True)

# Remove null values
df.dropna(inplace=True)

# Filter out rows where 'domain' contains 'Person'
df = df[~df['domain'].str.contains('Person')]

# Save the cleaned data to a new Excel file
df.to_excel('cleaned_data.xlsx', index=False)


In [8]:
df.save('cleaned_data.xlsx')

AttributeError: 'DataFrame' object has no attribute 'save'